# Create Agents to Research and Write an Article
Get foundational concepts of multi-agent systems and get an overview of the crewAI framework.

Install the following libraries

In [ ]:
%pip install crewai==0.1.32 langchain_community

Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

/Users/Roshan_Jangid/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/PythonScripts/pysparkenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


As a LLM for your agents, you'll be using OpenAI's gpt-3.5-turbo.
Optional Note: crewAI also allow other popular models to be used as a LLM for your Agents.

In [ ]:
import os
import getpass

# Prompt the user for input (hidden)
api_key = getpass.getpass("Enter the openai api key to set it as environment variable 'OPENAI_API_KEY': ")
api_base = getpass.getpass("Enter the openai api base to set it as environment variable 'OPENAI_API_BASE': ")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
topic = "Artificial Intelligence"
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
)

Populating the topic to update agents and tasks with the topic

In [ ]:
# Create a list of agents and tasks
agents = [planner, writer, editor]
tasks = [plan, write, edit]

# Define the topic
topic = "Artificial Intelligence"

# Function to update agents and tasks with the topic
def update_topic(agents, tasks, topic):
    for agent in agents:
        agent.goal = agent.goal.format(topic=topic)
        agent.backstory = agent.backstory.format(topic=topic)
    for task in tasks:
        task.description = task.description.format(topic=topic)

# Update all agents and tasks with the topic
update_topic(agents, tasks, topic)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff()

[DEBUG]: Working Agent: Content Planner
[INFO]: Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No

Final Answer: 

1. **Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence:**

   - **Trends:**
     - Deep Learning: A subset of machine learning that uses neural networks with many layers (deep structures). 
     - AI in Healthcare: From predicting patient risks to robotic surgeries, AI is revolutionizing healthcare.
     - AI and Big Data: AI is being used to process and analyze large volumes of data, driving data-driven decision making.
     - Ethical AI: As AI becomes more integrated into society, t

In [ ]:
from IPython.display import Markdown
Markdown(result)

The blog post is well-written and aligns with journalistic best practices. It provides balanced viewpoints, avoids major controversial topics, and is in line with the organization's writing style. However, there are a few minor edits to improve the text's clarity and coherence:

1. In the "Introduction" section, replace "From deep learning algorithms that can outperform human experts, to its revolutionary role in healthcare - AI is here and it’s transforming our world." with "From deep learning algorithms that outperform human experts to its revolutionary role in healthcare, AI is not just a part of our world—it's transforming it."

2. In the "Trends in AI" section, replace "Simultaneously, Ethical AI has become a crucial conversation, as we grapple with the implications of integrating AI into society and the need for regulation." with "At the same time, the conversation around Ethical AI is becoming crucial as we grapple with the implications of integrating AI into society and the impending need for regulation."

3. In the "Key Players in AI" section, replace "Not to be left out, OpenAI is striving to ensure that the benefits of artificial general intelligence are accessible to all." with "OpenAI, another notable player, is striving to ensure that the benefits of artificial general intelligence are accessible to all."

4. In the "AI News" section, replace "In the news, AI continues to make headlines." with "AI continues to dominate the news."

5. In the "Conclusion & Call to Action" section, replace "With AI shaping our present and future, the only question left is - are you ready for the AI revolution?" with "With AI shaping our present and future, the real question is—are you ready for the AI revolution?"